# Schilder Model Trainieren
Bevor wir beginnen importieren wir wieder einige Packages und testen ob wir eine GPU zur verfügung haben:

In [ ]:
from aic.helpers import disable_warnings
disable_warnings()

import random
from pathlib import Path

import torch
from torchvision.transforms import v2
import  matplotlib.pyplot as plt
import pandas as pd
import numpy as np

print(f'GPU (Cuda) is available: {torch.cuda.is_available()}')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('mps') if torch.mps.is_available() else torch.device('cpu')

## Augmentation

Beim trainieren von Machine-Learning-Modellen sind die Trainingsdaten oft der Hauptfaktor ob das Modell das gewünschte Wissen lernt oder nicht. Gleichzeitig ist das sammeln von Daten Zeitaufwändig (teuer) oder zum Teil sogar unmöglich (z.B. seltene Krankheit). Eine Technik um aus einem begrenzten Datenset mehr zu machen ist "Data Augmentation". In unserem Fall heisst das, dass wir verschiedene Operationen auf unsere gesammelten Bilder anwenden wie zum Beispiel spiegeln oder drehen um auf mehr Bildern zu trainieren. Wichtig ist dass die augmentationen die Bilder genug stark verändern damit die Bilder für das Modell nicht gleich aussehen - sonst lassen wir es besser bleiben.

Für die augmentation verwenden wir eine Funktion aus pytorch: `torchvision.transforms.v2.Compose[ ... ]`

Das folgende Beispiel zeigt eine der möglichen Augmentationen: `RandomRotation(degrees=(0, 180))`

![image.png](../assets/pytorch_random_rotation_illustration.png)

Die folgenden augmentationen (in pytorch transforms genannt) sind im datalaoder bereits implementiert:

In [ ]:
image_size = (320, 240)

transforms = v2.Compose([
    v2.RandomHorizontalFlip(p=0.2),
    v2.RandomRotation(degrees=30),
    # 📝 Deine Augmentationen
    v2.Resize(image_size[::-1]),            # skaliert das Bild zur Grösse die das Modell erwartet
    v2.ToImage(),                           # konvertiert das Bild zu einem torchivision.Image
    v2.ToDtype(torch.float32, scale=True),  # konvertiert und skaliert das Image zu einem Tensor
])

### 📝 Aufgabe 4.2
Ergänze / ändere die transforms um die Bilder stärker zu augmentieren. Benutz die [Übersicht](https://docs.pytorch.org/vision/main/auto_examples/transforms/plot_transforms_illustrations.html) und wähle die Augmentationen aus welche für unsere Schilder-Daten Sinn machen. Beachte dass gewisse Augmentationen die Bilder so verändern können, dass das Label nicht mehr stimmt und das Modell etwas falsches lernen würde.

## Dataloader

Wie beim Fahrmodell brauchen wir ein Python Klasse welche die Daten lädt. Der dataloader im `aic_util` package hat
folgende Funktionen:
- Lädt alle Bilder aus den Unterordner vom mitgegebenen Pfad wobei das Label dem Namen des Unterordners entspricht
- Mischt und balanciert die Bilder falls gewünscht (argument = `True`)
- [Augmentiert](https://docs.pytorch.org/vision/master/auto_examples/transforms/plot_transforms_getting_started.html) die Bilder (Spiegeln, verzerren, rotieren, farbanpassungen, unschärfe)
- Teilt die Bilder anhand des angebenden Verhältnisses auf ([train / validation split](https://www.v7labs.com/blog/train-validation-test-set))
- Konvertiert die Bilder zu Tensoren damit sie von unserem Machine Learning Modell verwendet werden können

### 📝 Aufgabe 4.3
Importiere die `load_dataset` Funktion von `aic_util.sign.dataloader` und lade ein test und ein eval
datenset von deinen gelabelten Daten. Printe die Grösse der beiden Datasets. Achtung: die `len` Funktion gibt
die Anzahl batches zurück, nicht die Anzahl Bilder.

Die Werte für `split_ratio` und `batch_size` sind sinnvolle default Werte. Während dem Training kannst du mit diesen
Werten herumspielen, finde aber zuerst heraus was sie genau machen.

In [ ]:
from aic.sign.dataloader import load_dataset, ImageDataset

data_path = Path('./data/phase-1')
split_ratio = 0.8  # wie viel % der Daten sollen im train vs im eval dataset verwendet werden
batch_size = 64  # wie viele Bilder in einem Batch (auf einmal) verarbeitet werden

# 📝 Deine Code


### 📝 Aufgabe 4.4
Bei Klassifizierung ist es wichtig dass wir gleich viele Bilder pro Klasse haben, sonst lernt das Modell
einfach immer die häufigste Klasse vorherzusagen. Die `load_data` Funktion macht das bereits für uns. Finde den
entsprechenden Parameter und setzte ihn oben auf den korrekten Wert.

## Auswertung
Bevor wir unser Modell trainieren brauchen wir noch einige Hilfsfunktionen um das Modell zu beurteilen und auszuwerten.
Überlege dir zuerst folgende Punkte:
1. Wie können wir die performance unseres Modells messen, gibt es eine oder mehrere Metriken welche relevant sind?
2. Wie können wir diese Metriken visualisieren und vergleichen?
3. Welche Zahlen wollen wir erreichen? Wann ist das Modell gut genug?

### Bilder darstellen
Als erstes erstellen wir eine Funktion welche aus einem Datenset einige zufällige Bilder anzeigt damit wir
kontrollieren können ob:
- Die richtigen Bilder geladen werden
- Die Klassen/Labels stimmen
- Die Augmentationen sinnvoll sind

Das Dataset gibt `torch.Tensor` zurück. Benutze die `tensor_to_image(...)` funktion um aus dem Tensor eine pixel array zu machen die mit `imshow(...)` anzeigbar ist.

### 📝 Aufgabe 4.5
Implementiere die Funktion:

In [ ]:
from aic.sign.helper import tensor_to_image


def show_sign_images(dataset: ImageDataset):
    labels = dataset.labels

    # 📝 Aufgabe: Implementiere die Funktion:


show_sign_images(train_data.dataset.dataset)

### Confusion Matrix

<Bild CM>

Ein weitverbreitetes Tool um die predictions (vorhersagen) eines Klassifikationsmodells darzustellen ist die
[Confusion Matrix](https://www.datacamp.com/de/tutorial/what-is-a-confusion-matrix-in-machine-learning).

### 📝 Aufgabe 4.6
Vervollständige die Funktion um eine Confusion Matrix zu plotten. Die Funktion erhält einen [Dataframe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) (eine Art Tabelle) mit den folgenden
Spalten:
- `image`:      Pfad zum Bild
- `true`:       Das wahre label
- `prediction`: Das vorhergesagte label
- `confidence`: Zahl zwischen 0 und 1 wie sicher sich das Modell ist
- `output`:     Der gesamte Output des Modells (alle Klassen + confidence) als Python dictionary

Speichere die Confusion Matrix an den entsprechenden Ort falls der Funktion ein Pfad mitgegeben wird.

_Optional:_ Die schwarzen Zahlen in den Kästchen sind bei gewissen Farben (z.B. Dunkelblau) schlecht lesbar. Ändere deine Funktion so, dass die Farbe der Zahlen sind ändert anhand des Hintergrunds. Tipp: Mit `cm.max().max() / 2` kann mittelpunkt aller Werte in der Confusion Matrix berechnet werden.

In [ ]:
def plot_confusion_matrix(df, save_to: Path | str | None):
    labels = ['50Sign', 'ClearSign', 'NoSign', 'StopSign'] # hardcoded
    cm = pd.DataFrame(0, index=labels, columns=labels)  # Eine leere Tabelle für unsere Confusion Matrix

    # 📝 Aufgabe: Vervollständige die Funktion:



example_df = pd.read_csv('../assets/example_model_output.csv')
plot_confusion_matrix(example_df, None)
# Die confusion Matrix sollte wie folgt aussehen:
#       2 0 0 1
#       0 2 0 0
#       0 0 2 0
#       1 0 0 2

## Modell

Jetzt sind wir bereit das Schilder-Modell zu trainieren. Unser Modell gliedert sich in zwei Teile:
1. **Feature Extractor:** Seine Aufgabe ist es aus den Pixel des Bildes Muster und Eigenschaften (sogenannte Features) zu lernen. Er besteht aus mehreren [CNN Layers](https://de.wikipedia.org/wiki/Convolutional_Neural_Network) hintereinander und ist sehr ähnlich zu den ersten Layers des Fahrmodells.
2. **Classifier:** Im Gegensatz zum Fahrmodell möchten wir jetzt ein Modell welches Klassen (unsere Schilder) vorhersagt. Dazu verwenden wir ein kleines lineares Netz welches aus den Features die Klasse vorhersagt.

### Vorgehen
1. Modell Architektur definieren
2. Trainingsdaten laden
3. Trainer erstellen und Modell trainieren
4. Auswerten und optimieren

Bevor wir beginnen importieren wir wieder einige Packages und testen ob wir eine GPU zur verfügung haben:

In [ ]:
from pathlib import Path
from typing import OrderedDict

import torch
from torch import nn
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelSummary
from matplotlib import pyplot as plt

from aic.sign.model import LightningModel
from aic.sign.dataloader import load_dataset
from aic.logger import DictLogger
from aic.sign.test import test_model
from aic.runs import create_run_dir

print(f'GPU (Cuda) is available: {torch.cuda.is_available()}')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('mps') if torch.mps.is_available() else torch.device('cpu')

### Modell Architektur
Das folgende Modell ist ein guter Startpunkt. Du darfst es gerne anpassen.

In [ ]:
class BaselineSignModel(nn.Module):
    def __init__(self, num_classes=4):
        super(BaselineSignModel, self).__init__()
        self.num_classes = num_classes

        self.layers = nn.Sequential(
            OrderedDict(
                conv1 = nn.Conv2d(3, 12, 3, 2, 1),
                batch1 = nn.BatchNorm2d(12),
                relu1 = nn.ReLU(),

                conv2 = nn.Conv2d(12, 16, 3, 2, 1),
                batch2 = nn.BatchNorm2d(16),
                relu2 = nn.ReLU(),

                conv3 = nn.Conv2d(16, 32, 3, 1, 1),
                batch3 = nn.BatchNorm2d(32),
                relu3 = nn.ReLU(),

                conv4 = nn.Conv2d(32, 32, 3, 1, 1),
                batch4 = nn.BatchNorm2d(32),
                relu4 = nn.ReLU(),

                pool = nn.MaxPool2d(2),

                conv5 = nn.Conv2d(32, 32, 3, 1, 1),
                batch5 = nn.BatchNorm2d(32),
                relu5 = nn.ReLU(),

                average_pooling = nn.AdaptiveAvgPool2d(1),

                flatten = nn.Flatten(),
                dropout = nn.Dropout(0.2),
                linear = nn.Linear(32, num_classes),
            )
        )

    def forward(self, x):
        return self.layers(x)


Vor dem Training erstellen wir einen Ordner in den wir das Modell sowie die Auswertung speichern können:

In [ ]:
run_directory = create_run_dir('../runs/sign/')

Um das Training zu vereinfachen, laden wir unser Pytorch Modell als Pytorch Lightning Modell. Zusätzlich definieren wir die Hyperparameter:

In [ ]:
EPOCHS = 25
LEARNING_RATE = 0.005
BATCH_SIZE = 64

model = BaselineSignModel(num_classes=4)
lightning_model = LightningModel(model, epochs=EPOCHS, lr=LEARNING_RATE)

logger = DictLogger()
trainer = Trainer(max_epochs=EPOCHS, callbacks=[logger])

ModelSummary().on_fit_start(Trainer(), lightning_model)

Jetzt sind wir bereit unser Modell zu trainieren:

In [ ]:
%%time
torch.set_float32_matmul_precision('medium')  # Beschleunigt das Training auf Kosten leicht geringerer Präzision

trainer.fit(lightning_model, train_data, eval_data)

### Training auswerten

### 📝 Aufgabe 4.7
Um das Training beurteilen zu können müssen wir die geloggten Metriken auswerten. In `logger.metrics` sind für jede Epoche folgende Metriken gespeichert: Validation Loss, Validation Accuracy, Training Loss und Training Accuracy. Plotte die 4 Metriken mit Matplotlib und vergiss nicht den Plot in der run directory zu speichern.

*Tipp: Es kann sein, dass nicht alle Metriken in allen Epochen geloggt werden.*

In [ ]:
# 📝 Aufgabe: Plotte die Metriken


### Modell testen
Die Trainingsmetriken zeigen uns schon auf wie gut unser Training geklappt hat. Um einen noch besseren Einblick zu bekommen wie gut das trainierte Modell tatsächlich ist, können wir es auf einem Dateset testen. Dazu verwendest du am besten Daten welche das Modell noch nicht gesehen hat, also Bilder die nicht während dem Training verwendet wurden.

In [ ]:
test_data = load_dataset('./data/test', batch_size=1, split_ratio=None, augment=False)
print(f'testing with {len(test_data)} images...')

metrics = test_model(lightning_model, test_data, device)

# 📝 Aufgabe: Verschiedene Tests implementieren, z.B:
#   - Confusion Matrix
#   - Bilder und Predictions manuell anschauen
#   - ...


### Modell Export
Um das Modell auf dem Auto laufen lassen zu können, müssen wir es exportieren. Dazu verwenden wir wieder das ONNX-Format.

In [ ]:
model_name = 'SignModel.onnx'
model_export_path = run_directory / model_name

batch = next(iter(train_data))
images, _, _ = batch
example_image = images[:1]  # onnx benötigt ein Beispielinput um das Modell exportieren zu können

with torch.no_grad():
    torch.onnx.export(
        model.to('cpu'),
        example_image,
        model_export_path,
        export_params=True,
        opset_version=17,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
    )

## Optimieren
Nachdem wir unser erstes Modell trainiert und getestet haben ist es wahrscheinlich noch nicht perfekt. Machine Learning lebt von einem iterativen Prozess der Optimierung. Jetzt geht es darum das Modell zu verbessern. Wir starten immer mit einem trainierten Modell, ändern etwas (Daten, Architektur, Parameter, ...), trainieren, testen und vergleichen. Dieser Prozess wird oft auch ein Experiment genannt. Zwischen zwei Experimenten sollte nicht zu viel verändert werden sonst ist es schwierig herauszufinden welche änderung was bewirkt, oder zwei änderungen heben sich gegenseitig auf.

- Hyperparameter
    - Learning Rate
    - Epochen
    - Batch Size
- Modell Architektur
    - Anzahl Convolution Layers (Feature Extractor)
    - Anzahl Linear Layers (Classifier)
    - Anzahl Channels in den Layers
    - Stride Grösse in den Layers
    - Häufigkeit der Pooling Operationen
- Verschiedene Augmentationen (ohne, mit, verschiedene transforms, verschieden stark)
- Unterschiedliche Datensets (grösser, kleiner)

Wichtig: Zum Optimieren betrachten wir 2 Aspekte.
1. Trainingsfortschritt:
    - Lernt das Modell überhaupt etwas? -> Architektur
    - Lernt das Modell genug schnell / zu schnell? -> Learning Rate
    - Hat das Modell schon lange fertig gelernt oder gab es am Schluss noch Verbesserungen? -> Epochen
    - Ist der Train-loss nahe bei null, der Validation-loss aber hoch oder steigt sogar? -> Overfitting
2. Modell Performance:
    - Confusion Matrix
    - Manuelle inspektion
    - Weitere Metriken, z.B. Accuracy (% richtige predictions), [F1](https://en.wikipedia.org/wiki/F-score)


**Tipps:**
- Grundsätzlich kann ein grösseres Modell (mehr Layers) mehr lernen. Falls die Anzahl der Trainingsdaten aber klein ist bringt es oft wenig das Modell zu vergrössern (overfitting). Dazu kommt, dass das Modell auf dem Auto (Raspberry Pi 4) laufen muss. Ein sehr grosses Modell (>100'000 Parameter) läuft vielleicht mit wenigen Frames pro Sekunde und kann daher nicht schnell genug auf die Linie reagieren.